Links helped us in this part:
1 https://netpbm.sourceforge.net/doc/pgm.html 
Explains what is a pgm file.
2 https://stackoverflow.com/questions/35723865/read-a-pgm-file-in-python
Explains what is the pgm file.
3 Now I'm facing a new issue which is the b character at the beginning of every line
https://www.geeksforgeeks.org/effect-of-b-character-in-front-of-a-string-literal-in-python/
4 Writing the data in a file for debugging
https://www.pythonmorsels.com/creating-and-writing-file-python/#:~:text=To%20write%20to%20a%20text,file%20to%20write%20to%20it.

In [1]:
def read_pgm(pgmf):
    """Return a raster of integers from a PGM as a list of lists."""
    assert pgmf.readline().decode('ascii') == 'P5\n'
    (width, height) = [int(i) for i in pgmf.readline().decode('ascii').split()]
    depth = int(pgmf.readline())
    assert depth <= 255

    raster = []
    for y in range(height):
        d = [];
        for y in range(width):
            d.append(ord(pgmf.read(1)))
        raster.append(d)
    return raster
data = []
label = []
for i in range(1,41):
    for j in range(1,11):
        f = open(f'J:/champions work/datasets/archive (1)/s{i}/{j}.pgm', 'rb')
        image = read_pgm(f)
        label.append(i)
        data.append(image)
num_images = len(data)
image_width = len(data[0])
image_height = len(data[0][0])

print(f"Number of images: {num_images}")
print(f"Width of images: {image_width}")
print(f"Height of images: {image_height}")

Number of images: 400
Width of images: 112
Height of images: 92


In [6]:
import numpy as np
import pandas as pd

Now we'll try to reduce the number of pixels by the method mentioned here:
https://stackoverflow.com/questions/48435977/reduce-number-of-pixels-to-obtain-low-resolution-image

In [2]:
def reduce_pixels(original_image):
    new_image = []
    l = 0
    for i in range(0, len(original_image), 3):
        new_image.append([])
        for j in range(0, len(original_image[0]), 3):
            s = 0
            for x in range(i, min(i + 3, len(original_image))):
                for y in range(j, min(j + 3, len(original_image[0]))):
                    s += original_image[x][y];
            new_image[l].append(s/49)
        l = l + 1
    return new_image
def red(data):
    reduced_pixels_data = []
    for i in range(400):
        reduced_pixels_data.append(reduce_pixels(data[i]))
    return reduced_pixels_data
    # Now we'll vectorize the matrix


In [3]:
def vectorize(reduced_pixels_data):
    reduced_pixels_data_vectorized = []
    for i in range(400):
        v = []
        for j in range(len(reduced_pixels_data[0])):
            for l in range(len(reduced_pixels_data[0][0])):
                v.append(reduced_pixels_data[i][j][l])
        reduced_pixels_data_vectorized.append(v)
    return reduced_pixels_data_vectorized

Dataframe constructor has a strange behaviour

In [32]:
def PCA(D, alpha):
    meu = np.mean(D, axis=0)
    z = D - meu
    z = np.around(z, decimals=2)
    E = z.T @ z
    print(len(E))
    print(len(E[0]))
    eigenvalues, eigenvectors = np.linalg.eigh(E)
    total_variance = sum(eigenvalues)
    eigen_pairs = [(eigenvalues[i], eigenvectors[:, i]) for i in range(len(eigenvalues))]
    sorted_eigen_pairs = sorted(eigen_pairs, key=lambda x: x[0], reverse=True)
    Ut = []
    s = 0
    for eigenvalue, eigenvector in sorted_eigen_pairs:
        s += eigenvalue
        Ut.append(eigenvector)
        if(s/total_variance >= alpha):
            break;
    Ut = np.array(Ut)
    P = Ut.T @ Ut
    print(len(Ut))
    print(len(Ut[0]))
    return P 

Links that helped us in this part:
1 How to split the matrix
https://www.mathworks.com/matlabcentral/answers/102465-how-do-i-extract-the-odd-and-even-rows-of-my-matrix-into-two-separate-matrices-and-reconstruct-it-ba
https://www.geeksforgeeks.org/python-list-slicing/
2 A dataframe or a python list
https://www.quora.com/What-is-the-difference-between-a-DataFrame-and-a-list-of-tuples-in-Python-Which-is-better-in-terms-of-performance#:~:text=DataFrames%20also%20have%20built%2Din,datasets%20with%20a%20simple%20structure.

In [51]:
D = red(data)
v = vectorize(D)
train_data = np.array(v[1::2][:])
test_data = np.array(v[0::2][:])
train_labels = np.array(label[1::2].copy())
test_labels = np.array(label[1::2].copy())
P = PCA(train_data, .99)
projected_train_data = (P @ train_data.T).T
projected_test_data = (P @ test_data.T).T

1178
1178
149
1178


In [52]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors=1) 
  
knn.fit(projected_train_data, train_labels) 
# Predict on dataset which model has not seen before 
predict = (knn.predict(projected_test_data)) 
print(predict)
print(test_labels) 
s = 0
for i in range(len(test_labels)):
    if(test_labels[i] == predict[i]):
        s += 1
print(s / len(test_labels) * 100)

[16  2  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9  9  9 10 10 10
 10 38 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18 18 18 18 19 19 19 19 36 20
 20  3 20 20 21 21 21 21 21 22 22 22 22 22 23 23 23 23 23 24 24 24 24 24
 25 25 25 25 25 26 26 26 26 26 27 27 27 27 27 28 28 28 28 28 29 29 29 29
 29 30 30 30 30 30 31 31 31 30 31 32 32 32 32 32 33 33 33 33 33 34 34 34
 34 34 40 35 35 35 35 36 36  7 36 36 37 37 37 37 37 38 38 38 38 38 39 39
 39 39 39 40 40  5  5  5]
[ 1  1  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4  4  5  5  5  5
  5  6  6  6  6  6  7  7  7  7  7  8  8  8  8  8  9  9  9  9  9 10 10 10
 10 10 11 11 11 11 11 12 12 12 12 12 13 13 13 13 13 14 14 14 14 14 15 15
 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18 18 18 18 19 19 19 19 19 20
 20 20 20 20 21 21 21 21 21 22 22 22 22 22 23 23 23 23 23 24 24 24 24 24
 25 25 25 25 25 26 26 26 